In [ ]:
import mlflow
### Q1. Install MLflow
!mlflow --version

### Q2. Download and preprocess the data
#### 4 files in ./output

mlflow, version 2.22.0


In [2]:
### Q3. Train a model with autolog
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("homework_2")

2025/05/25 03:41:23 INFO mlflow.tracking.fluent: Experiment with name 'homework_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748144483894, experiment_id='1', last_update_time=1748144483894, lifecycle_stage='active', name='homework_2', tags={}>

In [12]:
# train.py
import os
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run_train(data_path: str):
    with mlflow.start_run():
        mlflow.set_tag("Developer", "RenanGAS")

        mlflow.log_params({
            "train_data_path": "./data/train.pkl",
            "val_data_path": "./data/val.pkl"
        })
        X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
        X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

        rf = RandomForestRegressor(max_depth=10, random_state=0)
        mlflow.log_params(rf.get_params())

        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)

        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("RMSE", rmse)

run_train("./output")